<a href="https://colab.research.google.com/github/Jhonald73/TPproj_github/blob/main/W2S2_Activities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import userdata
from openai import OpenAI
import pandas as pd
from sklearn.datasets import load_iris


# Load key from Google Colab Secrets
api_key = userdata.get('OPENAI_API_KEY')


client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)

# Load the Iris dataset
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# Convert the first few rows to a string to send to OpenAI
data_preview = df.head(10).to_csv(index=False)
print(data_preview)

response = client.responses.create(
    model="gpt-4o-mini",
    instructions="You are an expert data scientist with extensive knowledge of predictive analysis and linear regression.",
    input=f"Dataset: Iris flower classification (150 samples, 4 features, 3 classes)\nHere are the first 10 rows of the dataset:\n{data_preview}]",
)
print(response.output_text)


sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
5.1,3.5,1.4,0.2
4.9,3.0,1.4,0.2
4.7,3.2,1.3,0.2
4.6,3.1,1.5,0.2
5.0,3.6,1.4,0.2
5.4,3.9,1.7,0.4
4.6,3.4,1.4,0.3
5.0,3.4,1.5,0.2
4.4,2.9,1.4,0.2
4.9,3.1,1.5,0.1

The Iris dataset is a classic dataset used in pattern recognition and machine learning. It consists of 150 samples of iris flowers, characterized by four features (sepal length, sepal width, petal length, and petal width) and three classes (Iris setosa, Iris versicolor, and Iris virginica).

### Overview of the Features

1. **Sepal Length (cm)**: Length of the sepal.
2. **Sepal Width (cm)**: Width of the sepal.
3. **Petal Length (cm)**: Length of the petal.
4. **Petal Width (cm)**: Width of the petal.

### First 10 Rows of the Dataset

| Sepal Length | Sepal Width | Petal Length | Petal Width |
|--------------|-------------|--------------|-------------|
| 5.1          | 3.5         | 1.4          | 0.2         |
| 4.9          | 3.0         | 1.4          | 0